In [1]:
import numpy as np
import random 
from functools import reduce
from scipy.stats import unitary_group
from scipy.sparse import csr_matrix, kron
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image

In [2]:
#Generating CNOTs
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

P0 = np.array([[1, 0], [0, 0]]) # projector onto |0><0|
P1 = np.array([[0, 0], [0, 1]]) # projector onto |1><1|

def kron_all(ops): # suggestion by ChatGPT
    return reduce(np.kron, ops)

def cnot_nqubit(n, control, target):
    ops1 = [I] * n
    ops2 = [I] * n

    ops1[control] = P0
    ops2[control] = P1
    ops2[target] = X

    return kron_all(ops1) + kron_all(ops2)
n = 4
control = 0
targets = [1, 2, 3]

CNOTS = [cnot_nqubit(n, control, t) for t in targets]

In [3]:
# defining rotation gates
def Ry(A):
    sn, cs = np.sin(A/2), np.cos(A/2)
    M = np.array([[cs,-sn],[sn, cs]])
    return M
def Rz(A):
    phi = np.exp(-1j*A/2)
    M = np.array([[phi,0],[0,np.conj(phi)]])
    return M
    
#Returns a length-2 array instead of 2x2 matrix. Faster, since it's diagonal. Good for Z rotations!
def Rz2(A):
    phi = np.exp(-1j*A/2)
    return np.array([phi, np.conj(phi)])
    
def Rz2t(A):
    phi = np.exp(-1j*A/2)
    return np.array([[phi], [np.conj(phi)]])

Z2 = np.array([1,-1])
Z2t = np.array([[1],[-1]])
    
# Parameterising the general unitary with rotation gates
def U(A0,A1,A2): # Arguments: Euler Angles
    M = Rz2t(A1) * Ry(A0) * Rz2(A2)
    return M

def DA_all(A0,A1,A2):
    R1 = Rz2t(A1)
    R0 = Ry(A0)
    R2 = Rz2(A2)
    R02 = R0 * R2
    R102 = R1 * R02
    U = R102
    dA0 = (-0.5j) * R1 * (Y @ R02)
    dA1 = (-0.5j) * Z2t * R102
    dA2 = (-0.5j) * R102 * Z2
    return (U, dA0, dA1, dA2)

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse

# 2 Unitaries per layer effectively
def parameters(layers): # layers = number of unitary layers, so no-of CNOTs = layers - 1
    A = np.zeros((2*layers,3))
    for i in range(2*layers):
        for j in range(3):
            A[i][j] = random.uniform(0.0, 4*np.pi) 
    return A


#Get the unitary created by these four single-qubit unitaries at layer i.
def U_4squ(A, i):
    U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
    U2 = U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
    if i%3==0:
        return (np.kron(np.kron(U1, U2), np.eye(4)))
    if i%3==1:
        return np.kron(np.kron(np.kron(U1, np.eye(2)),U2),np.eye(2))
    if i%3== 2: 
        return np.kron(np.kron(U1, np.eye(4)), U2)
    #return np.kron(np.kron(U1, U2), np.kron(U3, U4))

#Like `U_4squ` - But, optionally, skip the single unitary labelled j.
def U_4squ_skip(A, i, j):
    if j%2==0:
        U1 = np.eye(2)
        U2 = U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
    else:
        U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
        U2 = np.eye(2)
    if i%3==0:
        return np.kron(np.kron(U1, U2), np.eye(4))
    if i%3==1:
        return np.kron(np.kron(np.kron(U1, np.eye(2)),U2),np.eye(2))
    if i%3== 2: 
        return np.kron(np.kron(U1, np.eye(4)), U2)
        
#Given a 1-qubit unitary Ui on site s (0-3), expand it to a 4-qubit unitary.
def expand_on_site(Ui, i, j): # i - layer, j - qubit number
    if j%2==0:
        return np.kron(Ui, np.eye(8))
    else:
        if i%3==0:
            return np.kron(np.eye(2), np.kron(Ui, np.eye(4)))
        elif i%3 ==1:
            return np.kron(np.eye(4), np.kron(Ui, np.eye(2)))
        elif i%3 == 2:
            return np.kron(np.eye(8), Ui)
    
#Like `U_4squ`, but it's only the unitary at the one site j. That is, it's the single-qubit
#unitary, but expanded to 4-qubits.
def U_4squ_only(A, i, j):
    return expand_on_site(U(A[j][0],A[j][1],A[j][2]), i, j)

def circuit_structure(A, layers, j, U_goal): # qubit number    
    q = (j-1) // 2
    Uf = np.eye(16)
    Ub = np.eye(16)
    for it in range(q):
        Uf = Uf @ U_4squ(A, it) @ CNOTS[it%3]
    Uf = Uf @ U_4squ_skip(A, q, j-1)
    for ib in range(q+1, layers):
        Ub = Ub @ CNOTS[(ib-1)%3] @ U_4squ(A, ib)
    U_tbm = inverse(Uf) @ U_goal @ inverse(Ub) # U_tbm = rhs , U to be made
    return U_tbm

#The trace inner product of two matrices. Equal to np.trace(A @ inverse(B)), but faster
def trace_prod_inv(A, B):
    # return np.trace(A @ inverse(B))
    return np.conj(np.vdot(A, B))

In [4]:
#Replaces computing the gradient + gradual adjustment, with just rounding each goal directly to its nearest unitary
#and using that. Note: no "alpha" for learning rate anymore.
def gradient_descent3(layers,U_goal, A, iterations):
    nsu = 2*layers #(number of single qubit unitaries)
    #inner_iterations = int(iterations/10)
    #print(inner_iterations)
    stop_flag=0
    Uerr_array = []

    for i in range(iterations):
        if i%10==0:
            print("iteration",i)
        if stop_flag ==1:
            print("Truncation")
            break

        #Kepp running track of the error: inverse(Uf) * U_goal * inverse(Ub). We'll update this dynamically.
        Ub = U_4squ(A, 0)
        for l in range(1, layers):
            Ub = Ub @ CNOTS[(l-1)%3] @ U_4squ(A, l)
        Ui_err = U_goal @ inverse(Ub)
            
        for l in range(0, layers):            
            for s in [0,1]:
                j = 2*l+s
                
                # RHS
                Ui_err = Ui_err @ U_4squ_only(A, l, j)
                
                #Compute the partial trace of Ui, to leave just the `s` qubit part
                if s == 0:
                    Ui2 = np.trace(Ui_err.reshape(2,8, 2,8), axis1=1, axis2=3)
                else:
                    if l%3==0:
                        Ui2 = Ui_err.reshape(2,2,4, 2,2,4)
                        Ui2 = np.trace(Ui2, axis1=0, axis2=3)
                        Ui2 = np.trace(Ui2, axis1=1, axis2=3)
                    elif l%3==1:
                        Ui2 = Ui_err.reshape(4,2,2, 4,2,2)
                        Ui2 = np.trace(Ui2, axis1=0, axis2=3)
                        Ui2 = np.trace(Ui2, axis1=1, axis2=3)
                    elif l%3 == 2:
                        Ui2 = np.trace(Ui_err.reshape(8,2, 8,2), axis1=0, axis2=2)
                
                #Round goal to nearest unitary
                svd = np.linalg.svd(Ui2)
                unitized = svd[0] @ svd[2]
                unitized /= np.sqrt(np.linalg.det(unitized)) #Convert from U(2) to SU(2)

                # Get Euler angles
                beta = 2 * np.arctan(np.abs(unitized[1][0] / unitized[0][0]))
                phia = np.angle(unitized[0][0])
                phib = np.angle(unitized[1][0])
                a = phib - phia
                c = -(phia + phib)
                
                A[j][0]=beta
                A[j][1]=a
                A[j][2]=c
                
                #computing cost function
                # Uk = U(A0,A1,A2)
                Uk = unitized
                Tr = trace_prod_inv(Ui2, unitized)
                Abs = np.abs(Tr)
                Uerr = 16 - Abs
                if Uerr < 1e-8:
                    stop_flag=1
                    break
                
                
                Ui_err = inverse(expand_on_site(unitized, l,j)) @ Ui_err
                
            Ui_err = CNOTS[l%3] @ Ui_err @ CNOTS[l%3]
            if stop_flag==1:
                break
        Uerr_array.append(Uerr)
            
    return Uerr_array